# 🧠 Deep Learning: LSTM for Soccer Sequence Prediction

이 노트는 축구 경기의 **시퀀스(Sequence)** 데이터를 처리하기 위해 **LSTM (Long Short-Term Memory)** 모델을 구축합니다.
단편적인 상태만 보는 것이 아니라, **"패스 -> 드리블 -> 슛"**으로 이어지는 맥락(Context)을 읽어내어 마지막 패스의 도달 위치를 예측합니다.

### 🚀 Key Strategy
1. **Sequence Data Prep**: `game_episode` 단위로 데이터를 묶어 시계열 입력(3D Tensor)으로 변환합니다.
2. **Embedding Layers**: `player_id`, `type_name` 등 범주형 변수를 임베딩 벡터로 변환하여 학습합니다.
3. **LSTM Model**: 시간의 흐름에 따른 패턴을 학습합니다.

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
import gc

# GPU 확인
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## 1. Data Loading & Preprocessing

In [ ]:
PREPROCESS_DIR = './data_preprocess'
OPEN_DIR = './open_track1'

# 1. Load Train (Feature Merged)
# 우리가 만든 'train_final.csv'에는 player_stats가 포함되어 있습니다.
train_df = pd.read_csv(os.path.join(PREPROCESS_DIR, 'train_final.csv'))

# 2. Load Test (Full Sequence)
# 주의: 'test_final.csv'는 마지막 행만 저장했으므로 LSTM용으로는 부적합합니다.
# 다시 원본 테스트 파일들을 읽어서 시퀀스를 만들어야 합니다.
test_index = pd.read_csv(os.path.join(OPEN_DIR, 'test.csv'))
player_stats = pd.read_csv(os.path.join(PREPROCESS_DIR, 'player_features.csv')) # Player stats 다시 로드

# 테스트 데이터 로드 함수
def load_full_test_data(test_index, player_stats):
    test_list = []
    for idx, row in tqdm(test_index.iterrows(), total=len(test_index), desc="Loading Test Sequences"):
        file_path = os.path.join(OPEN_DIR, row['path'])
        df = pd.read_csv(file_path)
        
        # Player Stats 병합
        df = pd.merge(df, player_stats, on='player_id', how='left')
        
        # 식별자 추가
        df['game_episode'] = row['game_episode']
        
        test_list.append(df)
    
    return pd.concat(test_list, ignore_index=True)

print("Loading Test Data...")
test_df = load_full_test_data(test_index, player_stats)

# 결측치 채우기 (Train과 동일하게)
fill_values = {
    'avg_pass_dist': player_stats['avg_pass_dist'].mean(),
    'avg_pass_angle': player_stats['avg_pass_angle'].mean(),
    'pass_count': 0
}
test_df.fillna(fill_values, inplace=True)
train_df.fillna(fill_values, inplace=True)

print(f"Train Shape: {train_df.shape}, Test Shape: {test_df.shape}")

## 2. Sequence Generation (시퀀스 변환)
LSTM 학습을 위해 `(Samples, Timesteps, Features)` 형태의 3D 텐서로 변환합니다.

In [ ]:
# 사용할 변수 정의
NUMERIC_COLS = ['time_seconds', 'start_x', 'start_y', 'avg_pass_dist', 'avg_pass_angle']
CAT_COLS = ['player_id', 'type_name'] # 임베딩할 변수
TARGET_COLS = ['end_x', 'end_y']

# 1. Scaling (수치형 변수)
scaler = StandardScaler()
train_df[NUMERIC_COLS] = scaler.fit_transform(train_df[NUMERIC_COLS])
test_df[NUMERIC_COLS] = scaler.transform(test_df[NUMERIC_COLS])

# 타겟 스케일링 (0~105 -> 0~1로 하면 학습 안정성 증가)
train_df['end_x'] = train_df['end_x'] / 105.0
train_df['end_y'] = train_df['end_y'] / 68.0
# 테스트는 타겟이 NaN이므로 패스

# 2. Encoding (범주형 변수)
for col in CAT_COLS:
    le = LabelEncoder()
    # Train과 Test의 모든 값을 포함하여 피팅
    all_vals = pd.concat([train_df[col], test_df[col]]).astype(str).unique()
    le.fit(all_vals)
    
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))
    
    print(f"Encoded {col}: {len(le.classes_)} classes")

# 3. Sequence Padding Function
MAX_LEN = 20 # 시퀀스 최대 길이 (분석 결과 평균이 작다면 줄여도 됨)

def create_sequences(df, is_train=True):
    # game_episode 별로 그룹화
    grouped = df.groupby('game_episode')
    
    seq_numeric = []
    seq_cat1 = [] # player_id
    seq_cat2 = [] # type_name
    targets = []
    ids = []
    
    for name, group in tqdm(grouped, desc="Creating Sequences"):
        # 시간순 정렬 (이미 되어있겠지만 확실히)
        # group = group.sort_values('time_seconds') # time_seconds가 스케일링되어 있어도 정렬은 유지됨
        
        # Feature 추출
        num_vals = group[NUMERIC_COLS].values
        cat1_vals = group['player_id'].values
        cat2_vals = group['type_name'].values
        
        # Train인 경우: 마지막 행의 타겟을 가져옴
        # 하지만 입력 시퀀스에는 '마지막 행'까지 포함해야 하는가, 아니면 '마지막 행 직전'인가?
        # 문제 정의: "주어진 시퀀스의 마지막 패스 도착 좌표"
        # 즉, 마지막 행(=그 패스 자체)의 start_x, start_y, action_type 등은 알고 있음.
        # 따라서 입력 시퀀스 = [..., 이전_행, 마지막_행]
        
        seq_numeric.append(num_vals)
        seq_cat1.append(cat1_vals)
        seq_cat2.append(cat2_vals)
        ids.append(name)
        
        if is_train:
            # 마지막 행의 end_x, end_y가 타겟
            # 주의: group 순서가 섞이지 않았다면 iloc[-1]이 맞음
            target = group.iloc[-1][TARGET_COLS].values
            targets.append(target)
            
    # Padding
    # pre or post? LSTM은 보통 'post' padding 후 Masking 레이어 사용하거나, 'pre' padding 사용.
    # 최근 흐름은 'pre' (뒤쪽 값에 집중)가 좋다고 함.
    
    X_num = pad_sequences(seq_numeric, maxlen=MAX_LEN, padding='pre', dtype='float32')
    X_cat1 = pad_sequences(seq_cat1, maxlen=MAX_LEN, padding='pre', dtype='int32')
    X_cat2 = pad_sequences(seq_cat2, maxlen=MAX_LEN, padding='pre', dtype='int32')
    
    if is_train:
        y = np.array(targets, dtype='float32')
        return [X_num, X_cat1, X_cat2], y, ids
    else:
        return [X_num, X_cat1, X_cat2], ids

# Create Train Sequences
X_train, y_train, train_ids = create_sequences(train_df, is_train=True)

# Create Test Sequences
X_test, test_ids = create_sequences(test_df, is_train=False)

print(f"X_train shape: {X_train[0].shape}")
print(f"y_train shape: {y_train.shape}")

## 3. Build & Train LSTM Model

In [ ]:
def build_model(num_len, cat1_vocab, cat2_vocab, emb_dim=16):
    # Inputs
    input_num = Input(shape=(None, num_len), name='numeric_in')
    input_cat1 = Input(shape=(None,), name='player_in')
    input_cat2 = Input(shape=(None,), name='type_in')
    
    # Embeddings
    emb_cat1 = Embedding(input_dim=cat1_vocab, output_dim=emb_dim, mask_zero=True)(input_cat1)
    emb_cat2 = Embedding(input_dim=cat2_vocab, output_dim=8, mask_zero=True)(input_cat2)
    
    # Concatenate: (Batch, Time, Features)
    # Num input도 Masking 처리가 필요할 수 있지만, 보통 concat 후 LSTM이 처리.
    # Masking Layer를 명시적으로 둘 수도 있음.
    
    concat = Concatenate()([input_num, emb_cat1, emb_cat2])
    
    # LSTM
    x = LSTM(64, return_sequences=True)(concat)
    x = LSTM(32)(x)
    x = Dropout(0.2)(x)
    
    # Output
    output = Dense(2, activation='linear')(x) # x, y for normalized targets
    
    model = Model(inputs=[input_num, input_cat1, input_cat2], outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

# Model Params
num_features = X_train[0].shape[2]
vocab_player = train_df['player_id'].max() + 1
vocab_type = train_df['type_name'].max() + 1

model = build_model(num_features, vocab_player, vocab_type)
model.summary()

In [ ]:
# Training with Validation Loop (간단히 holdout or k-fold)
BATCH_SIZE = 64
EPOCHS = 10 
# 시간 관계상 Epoch을 적게 잡았지만, 실제로는 EarlyStopping이 멈출 때까지 해야 함.

checkpoint = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1)

# Split Train/Val (based on IDs to avoid leakage)
# 간단하게 slicing
val_ratio = 0.2
val_cnt = int(len(X_train[0]) * val_ratio)

X_train_fold = [x[:-val_cnt] for x in X_train]
y_train_fold = y_train[:-val_cnt]
X_val_fold = [x[-val_cnt:] for x in X_train]
y_val_fold = y_train[-val_cnt:]

history = model.fit(
    X_train_fold, y_train_fold,
    validation_data=(X_val_fold, y_val_fold),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint, reduce_lr],
    verbose=1
)

## 4. Inference & Submission
학습된 모델로 테스트 데이터의 좌표를 예측합니다.

In [ ]:
preds = model.predict(X_test, batch_size=BATCH_SIZE)

# 스케일링 복구 (0~1 -> 0~105)
preds[:, 0] = preds[:, 0] * 105.0
preds[:, 1] = preds[:, 1] * 68.0

# Clipping (경기장 밖으로 나가지 않게 보정)
preds[:, 0] = np.clip(preds[:, 0], 0, 105)
preds[:, 1] = np.clip(preds[:, 1], 0, 68)

# 제출 파일 생성
submission = pd.DataFrame({
    'game_episode': test_ids,
    'end_x': preds[:, 0],
    'end_y': preds[:, 1]
})

# 정렬 (제출은 sample_submission 순서와 상관없을 수 있으나 맞추는게 좋음)
# sample_submission 로드
sample_sub = pd.read_csv('./open_track1/sample_submission.csv')
# game_episode 기준으로 병합하여 순서 맞춤
final_sub = pd.merge(sample_sub[['game_episode']], submission, on='game_episode', how='left')

save_path = 'submission_lstm.csv'
final_sub.to_csv(save_path, index=False)
print(f"Saved Submission: {save_path}")
final_sub.head()